In [1]:
import json
import requests
import urllib 
import subprocess
import os
import datetime
from datetime import date
from random import randint
from sqlite3 import Date
import pandas as pd
import numpy as np
import random
import re
from itertools import cycle


In [ ]:
#declare the information needed to access the server

account='your login'
passwd='your password?'
URL = 'your url'
ping = requests.get(URL+'/ping')
if ping.status_code == 200:
    try:
        data = ping.json()
        print(data)
    except json.JSONDecodeError:
        print("the request isn'it in the json format.")
else:
    print(f"request error with : {ping.status_code}.")


login= requests.post(URL+'/login', json={"identifier":account,"password":passwd})
token=login.json()['token']
print(token)

In [ ]:
### To see all tables attached to a Data Modul.
### In our systeme a Data Modul can contain severel tables, and we have teh REST API to check the Data Moduls. This API is Get/modul/modul_name/definition 
### In the place of sending this API by Insomnia or Postman we will use 'request' library of Python in this script.


module_name = input("Veuillez entrer le nom du module : ")
module_definition = requests.get(f'{URL}/module/{module_name}/definition',  headers={'Authorization': f'{token}'})
if module_definition.status_code == 200 :
    fact_table = module_definition.json().get('tables')
    print(f'fact_table : {fact_table}')    
        
    zips = []
    x = module_definition.json().get('arrays')
    for i in range(len(x)):
        zip = x[i].get('zipTable', {})  
        zips.append(zip)
    print ( f"zips : {zips}" )
        
    dictionaries = module_definition.json().get('dictionaries', {})
    if dictionaries:
       dico = set(dictionaries.values())
       print(f'dico : {dico}')
    else :
       dico = []
       print(f'No dico')
       
    facet = module_definition.json().get('facets', {})
    print(f'facet : {facet}')

       
else:
    print(f"ERROR")   
    

#ingestion_tables = zips + list(dico) + fact_table
ingestion_tables = list(dico) + fact_table
zip_dico = zips + list(dico)  
print(f'nb_table de ccr_ingestion_tables: {len(ingestion_tables)} : {ingestion_tables}')
print(zip_dico)

In [ ]:
#Here we will find the columns and theirs count for each table 

for table_name in ingestion_tables:
    ingest_settings = requests.get(f'{URL}/module/{module_name}/tables/{table_name}/ingestion_settings', headers={'Authorization': f'{token}'})
    if ingest_settings.status_code == 200:
        insertquery = ingest_settings.json().get('insertQuery')
        match = re.search(r"input\('(.+?)'\)", insertquery)
        my_string = match.group(1)
    else:
        print(f"ERROR")

    fields = []
    for elemt in my_string.split(','):
        elemt = elemt.strip('\'').lstrip(' ')
        fields.append(elemt.split(" "))

    variable_types_count = {}  
    total_occurrences = 0  

    column_names = [] 

    for elmt in fields:
        column_name = elmt[0]  
        column_type = elmt[1]  

        variable_types_count[column_type] = variable_types_count.get(column_type, 0) + 1
        total_occurrences += 1

        column_names.append(column_name)

    print(f'In the table: {table_name}')
    print("Columns'name:")
    for column_name in column_names:
        print(column_name)
    for v_type, count in variable_types_count.items():
        
        print(f"{v_type}: {count}")
    print(f"column count: {total_occurrences}")
    print("\n")

In [ ]:
# Here we will check the joins between the fact table and its dictionnaries

datamodel = requests.get(f'{URL}/datamodel/{module_name}',  headers={'Authorization': f'{token}'})
joins = datamodel.json().get('rawDataModel').get('joins')  #rwdatamodel denn sonra {} geldiyi ucun day [0] yazmiriq, [] gelse idi yazardiq
factkey = []
dicokey=[]
for i in range(len(joins)) :
    factkey_value = joins[i].get('key1').get('key')[0]
    factkey.append(factkey_value)
    dicokey_value = joins[i].get('key2').get('key')[0]
    dicokey.append(dicokey_value)
print(factkey)
print(dicokey)

In [ ]:
# This is the final step of script and also is crucial step as the main idea is to generate the data and ingest them into data base.

for table_name in ingestion_tables :
    ingest_settings = requests.get(f'{URL}/module/{module_name}/tables/{table_name}/ingestion_settings',  headers={'Authorization': f'{token}'})
    # here we will get the body of each table: I mean, we will have tha list of columns and theirs clickhouse types (for example, column_1 (String), column_2(float64) etc.)
    if ingest_settings.status_code == 200:
        insertquery = ingest_settings.json().get('insertQuery')
        match = re.search(r"input\('(.+?)'\)", insertquery)
        my_string= match.group(1)
    else:
        print(f"ERROR OF INSERTQUERY")
    
    fields = []
    for elemt in my_string.split(','):
        elemt = elemt.strip('\'').lstrip(' ')
        fields.append(elemt.split(" "))

    
    N = 10  # this is the number of rows for each csv file
    startdate=datetime.date(2022,4,20)
    datavec=[]
    datevec=[str(startdate-datetime.timedelta(days=i)) for i in range(0,500)]
    datavec=[random.sample(datevec, 250) for i in range(2)]
    df = pd.DataFrame()

    
    output_folder = module_name
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    #once we have all info about the body of tables, we will generate the data based on this info by runnig this for step:
    
    for itt in range(3,5): #btw, ths is for  Date, we will use it in the Rest API which is for ingestion. Actually, for each ingestion I'll use different dates. So, I'll ingest the data two times for each time. 1e is for the date 03, and second is for the 04 
        for elmt in fields[:]:
            if('String' in elmt[1] and 'Array' not in elmt[1]):
                df[elmt[0]] = [elmt[0]+str(random.randint(3000,5000)) for i in range(N)]
                df[elmt[0]] = ''+ df[elmt[0]] + 'berlin'
            if('String' in elmt[1] and 'Array'  in elmt[1]):
                df[elmt[0]] = [[elmt[0]+str(random.randint(3000,5000)) for i in range(3)] for i in range(N)]
            
            if ('String' not in elmt[1] and 'Array' in elmt[1]):
                if ('Float' in elmt[1]):
                    df[elmt[0]] = np.random.uniform(-399.5,399.5,(N,3)).tolist()
                if('Int8' in elmt[1]):
                    df[elmt[0]] = np.random.randint(-100,100,(N,3)).tolist()
                if('Int32' in elmt[1]):
                    df[elmt[0]] = np.random.randint(-10000,10000,(N,3)).tolist()
                if('Int64' in elmt[1]):
                    df[elmt[0]] = np.random.randint(-100000,100000,(N,3)).tolist()
                if('UInt8' in elmt[1]):
                    df[elmt[0]] = np.random.randint(0,100,(N,3)).tolist() 
                if('UInt64' in elmt[1]):
                    df[elmt[0]] = np.random.randint(0,1000000,(N,3)).tolist()
                if('UInt32' in elmt[1]):
                    df[elmt[0]] = np.random.randint(0,100000,(N,3)).tolist()                    
                if 'Nullable(Int)' in elmt[1]:
                    df[elmt[0]] = ''        
                if 'Nullable(Float)' in elmt[1]:
                    df[elmt[0]] = '' 
                    
            if ('String' not in elmt[1] and 'Array' not in elmt[1]):
                if('Float' in elmt[1]):
                    df[elmt[0]] = np.random.uniform(-399.5,399.5,N).tolist()
                if('Int8' in elmt[1]):
                    df[elmt[0]] = np.random.randint(-100,100,N).tolist()
                if('Int32' in elmt[1]):
                    df[elmt[0]] = np.random.randint(-10000,10000,N).tolist()
                if('Int64' in elmt[1]):
                    df[elmt[0]] = np.random.randint(-100000,100000,N).tolist()
                if('UInt8' in elmt[1]):
                    df[elmt[0]] = np.random.randint(0,10000,N).tolist() 
                if('UInt64' in elmt[1]):
                    df[elmt[0]] = np.random.randint(0,100000,N).tolist()
                if('UInt32' in elmt[1]):
                    df[elmt[0]] = np.random.randint(0,1000000,N).tolist()                    
                if 'Nullable(Int)' in elmt[1]:
                    df[elmt[0]] = ''        
                if 'Nullable(Float)' in elmt[1]:
                    df[elmt[0]] = ''       
            
        
            if('Date' in elmt[1] and 'DateTime' not in elmt[1]):
                df[elmt[0]] = [date.today() for i in range(N)]
            if('DateTime' in elmt[1]):
                df[elmt[0]] = [datetime.datetime.now() for i in range(N)]        
        join_1 = cycle(['L1composite', 'L2composite', 'L3composite', 'L4composite', 'L5composite'])
        join_2= cycle(['R1composite', 'R2composite', 'R3composite', 'R4composite', 'R5composite'])
        join_3 = cycle(['B1composite', 'B2composite', 'B3composite', 'B4composite', 'B5composite'])
        join_4 = cycle(['111', '112', '113', '114', '115'])
        join_5 = cycle(['511', '521', '531', '541', '551'])
        
        if 'join_1' in df:
            df['join_1'] = [next(join_1) for _ in range(len(df))]
        if 'join_2' in df:
            df['join_2'] = [next(join_2) for _ in range(len(df))]
        if 'join_3' in df:
            df['join_3'] = [next(join_3) for _ in range(len(df))]
        if 'join_4' in df :
            df['join_4'] = [next(join_4) for _ in range(len(df))]
        if 'join_5' in df :
            df['join_5'] = [next(join_5) for _ in range(len(df))]
            
        csv_path = os.path.join(output_folder, f"{table_name}_{itt}.csv")
        df.to_csv(csv_path, header=True, index=False)
        print(f'{table_name}_{itt}.csv a été créé')

        # After preparing the data, we will start ingestion by the Rest API 
        # before we will ingest the data into zip and dictionnary tables:        
        if table_name in zip_dico :
               
                dico_ingestion = f'{URL}/tables/{table_name}/2024-04-{itt:02}' ### :02 just pour avoir les dates comme 01, 02 etc
                headers = {
                'Authorization': f'{token}',
                'Content-Type': 'text/csv;header=present'
                    }

                with open(csv_path, 'rb') as file:
                    data = file.read()

                response = requests.post(dico_ingestion, headers=headers, data=data, verify=False)
    
                if response.status_code == 204:
                    print(f"for {table_name} le fichier {table_name}_{itt} a été ingeré")
                else:
                    print(f"Error in {table_name}: {response.status_code}, {response.text}")

        # and in the final part the fact table(main table) will be ingested

        elif table_name in fact_table:
            transactionId = requests.post(f'{URL}/tables/{table_name}/transaction/2024-04-{itt:02}/from/official/ingestion',  headers={'Authorization': f'{token}'})
            transactionId= transactionId.content.decode('utf-8')
            Fact_table_ingest = f'{URL}/tables/{table_name}/commits/{transactionId}/2024-04-{itt:02}' ### :02 just pour avoir les dates comme 01, 02 etc
            headers = {
                'Authorization': f'{token}',
                'Content-Type': 'text/csv;header=present'
                }

            with open(csv_path, 'rb') as file:
                data = file.read()

            response = requests.post(Fact_table_ingest, headers=headers, data=data, verify=False)
            if response.status_code == 200:
                closetransaction = f"{URL}/tables/{table_name}/closeTransaction/{transactionId}/2024-04-{itt:02}/to/official"

                headers = {
            "Authorization": token
                 }

                response = requests.post(closetransaction, headers=headers, verify=False)
                print(f"for {table_name} le fichier {table_name}_{itt} a été ingeré : {response}")
    
            else:
                print(f"Error for {table_name} : {response.status_code}, {response.text}")
            
            
        
        
        